<a href="https://colab.research.google.com/github/YimingXu1/DS-notes/blob/main/Projects/U.S.%20Patent%20Phrase%20to%20Phrase%20Matching/PatentSBERTa_train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This notebook is a stencil for bert model training

# Directory Setting

In [1]:
import os
from google.colab import drive
drive.mount('/content/drive')

path = "/content/drive/My Drive"

Mounted at /content/drive


In [2]:
!pip3 install datasets
!pip3 install transformers
!pip3 install sentencepiece
!pip3 install -U scikit-learn

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 346 kB 5.1 MB/s 
     |████████████████████████████████| 86 kB 5.4 MB/s 
     |████████████████████████████████| 1.1 MB 48.5 MB/s 
     |████████████████████████████████| 212 kB 64.1 MB/s 
     |████████████████████████████████| 140 kB 10.9 MB/s 
     |████████████████████████████████| 86 kB 4.7 MB/s 
     |████████████████████████████████| 596 kB 62.1 MB/s 
     |████████████████████████████████| 127 kB 64.3 MB/s 
     |████████████████████████████████| 271 kB 69.2 MB/s 
     |████████████████████████████████| 94 kB 3.0 MB/s 
     |████████████████████████████████| 144 kB 68.9 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
     

In [3]:
import os
import pandas as pd
import numpy as np
from sklearn.model_selection import KFold, StratifiedKFold
import shutil

import torch
from torch.utils.data import DataLoader, Dataset
import datasets, transformers
from transformers import TrainingArguments, Trainer
from transformers import AutoModelForSequenceClassification, AutoTokenizer

os.environ["WANDB_DISABLED"] = "true"

# CFG

In [4]:
class CFG:
    input_path = './drive/MyDrive/Kaggle/USPatent/uspatent/'
    #model_path = './drive/MyDrive/Kaggle/deberta-v3-small/'
    model_path =   'AI-Growth-Lab/PatentSBERTa' #'./drive/MyDrive/Kaggle/USPatent/Models/'
    
    learning_rate = 2e-5
    weight_decay = 0.01
    num_fold = 5
    epochs = 5
    batch_size =128
    max_input_length = 130
    batch_size = 64
    num_workers = 2



# Preprocessing

In [5]:
# https://www.kaggle.com/code/abhishek/phrase-matching-folds
def create_folds(data, num_splits):
    # we create a new column called kfold and fill it with -1
    data["fold"] = -1
    
    # the next step is to randomize the rows of the data
    # data = data.sample(frac=1).reset_index(drop=True)

    # calculate number of bins by Sturge's rule
    # I take the floor of the value, you can also
    # just round it
    # num_bins = int(np.floor(1 + np.log2(len(data))))
    
    # bin targets
    data.loc[:, "bins"] = pd.cut(
        data["score"], bins=5, labels=False
    )
    
    # initiate the kfold class from model_selection module
    kf = StratifiedKFold(n_splits=num_splits, shuffle=True, random_state=42)
    
    # fill the new kfold column
    # note that, instead of targets, we use bins!
    for f, (t_, v_) in enumerate(kf.split(X=data, y=data.bins.values)):
        data.loc[v_, 'fold'] = f
    
    # drop the bins column
    data = data.drop("bins", axis=1)

    # return dataframe with folds
    return data

In [6]:
train_df = pd.read_csv(f"{CFG.input_path}train.csv")
titles = pd.read_csv(f"{CFG.input_path}titles.csv")
train_df = train_df.merge(titles, left_on='context', right_on='code')
# ====================================================
# CPC Data
# ====================================================
cpc_texts = torch.load("./drive/MyDrive/Kaggle/folds-dump-the-two-paths-fix/cpc_texts.pth")
train_df['context_text'] = train_df['context'].map(cpc_texts)
train_df['text'] = train_df['anchor'] + '[SEP]' + train_df['target'] + '[SEP]'  + train_df['context_text']
train_df['text'] = train_df['text'].apply(str.lower)
display(train_df.head())

,id,anchor,target,context,score,code,title,section,class,subclass,group,main_group,context_text,text
0,37d61fd2272659b1,abatement,abatement of pollution,A47,0.50,A47,FURNITURE; DOMESTIC ARTICLES OR APPLIANCES; CO...,A,47.0,NaN,NaN,NaN,HUMAN NECESSITIES. FURNITURE; DOMESTIC ARTICLE...,abatement[sep]abatement of pollution[sep]human...
1,7b9652b17b68b7a4,abatement,act of abating,A47,0.75,A47,FURNITURE; DOMESTIC ARTICLES OR APPLIANCES; CO...,A,47.0,NaN,NaN,NaN,HUMAN NECESSITIES. FURNITURE; DOMESTIC ARTICLE...,abatement[sep]act of abating[sep]human necessi...
2,36d72442aefd8232,abatement,active catalyst,A47,0.25,A47,FURNITURE; DOMESTIC ARTICLES OR APPLIANCES; CO...,A,47.0,NaN,NaN,NaN,HUMAN NECESSITIES. FURNITURE; DOMESTIC ARTICLE...,abatement[sep]active catalyst[sep]human necess...
3,5296b0c19e1ce60e,abatement,eliminating process,A47,0.50,A47,FURNITURE; DOMESTIC ARTICLES OR APPLIANCES; CO...,A,47.0,NaN,NaN,NaN,HUMAN NECESSITIES. FURNITURE; DOMESTIC ARTICLE...,abatement[sep]eliminating process[sep]human ne...
4,54c1e3b9184cb5b6,abatement,forest region,A47,0.00,A47,FURNITURE; DOMESTIC ARTICLES OR APPLIANCES; CO...,A,47.0,NaN,NaN,NaN,HUMAN NECESSITIES. FURNITURE; DOMESTIC ARTICLE...,abatement[sep]forest region[sep]human necessit...


In [7]:
train_df = create_folds(train_df, CFG.num_fold)

In [8]:
train_df.head()

,id,anchor,target,context,score,code,title,section,class,subclass,group,main_group,context_text,text,fold
0,37d61fd2272659b1,abatement,abatement of pollution,A47,0.50,A47,FURNITURE; DOMESTIC ARTICLES OR APPLIANCES; CO...,A,47.0,NaN,NaN,NaN,HUMAN NECESSITIES. FURNITURE; DOMESTIC ARTICLE...,abatement[sep]abatement of pollution[sep]human...,1
1,7b9652b17b68b7a4,abatement,act of abating,A47,0.75,A47,FURNITURE; DOMESTIC ARTICLES OR APPLIANCES; CO...,A,47.0,NaN,NaN,NaN,HUMAN NECESSITIES. FURNITURE; DOMESTIC ARTICLE...,abatement[sep]act of abating[sep]human necessi...,3
2,36d72442aefd8232,abatement,active catalyst,A47,0.25,A47,FURNITURE; DOMESTIC ARTICLES OR APPLIANCES; CO...,A,47.0,NaN,NaN,NaN,HUMAN NECESSITIES. FURNITURE; DOMESTIC ARTICLE...,abatement[sep]active catalyst[sep]human necess...,2
3,5296b0c19e1ce60e,abatement,eliminating process,A47,0.50,A47,FURNITURE; DOMESTIC ARTICLES OR APPLIANCES; CO...,A,47.0,NaN,NaN,NaN,HUMAN NECESSITIES. FURNITURE; DOMESTIC ARTICLE...,abatement[sep]eliminating process[sep]human ne...,3
4,54c1e3b9184cb5b6,abatement,forest region,A47,0.00,A47,FURNITURE; DOMESTIC ARTICLES OR APPLIANCES; CO...,A,47.0,NaN,NaN,NaN,HUMAN NECESSITIES. FURNITURE; DOMESTIC ARTICLE...,abatement[sep]forest region[sep]human necessit...,0


In [9]:
tokenizer = AutoTokenizer.from_pretrained(CFG.model_path)

Downloading:   0%|          | 0.00/440 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/239 [00:00<?, ?B/s]

# Dataset

In [10]:
class TrainDataset(Dataset):
    def __init__(self, df):
        self.inputs = df['text'].values.astype(str)
        self.targets = df['target'].values.astype(str)
        self.label = df['score'].values

    def __len__(self):
        return len(self.inputs)

    def __getitem__(self, item):
        inputs = self.inputs[item]
        targets = self.targets[item]
        label = self.label[item]
        
        
        inputs = tokenizer(inputs, 
                    targets,
                    max_length=64,
                    padding='max_length',
                    truncation=True )
        return {**inputs,
               'labels':torch.as_tensor(label, dtype=torch.float) }

# Fine-tuning the model

In [11]:
from datasets import load_metric
metric = load_metric('glue', 'stsb')
metric

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    return metric.compute(predictions=predictions, references=labels)

In [12]:
oof_df = pd.DataFrame()
model = AutoModelForSequenceClassification.from_pretrained(CFG.model_path, num_labels=1)

metric_name = "pearson"
model_name = CFG.model_path.split("/")[-1]
    
args = TrainingArguments(
    output_dir=f"./drive/MyDrive/Kaggle/USPatent/{model_name}-finetuned",
    evaluation_strategy = "epoch",
    save_strategy = "epoch",
    learning_rate=CFG.learning_rate,
    per_device_train_batch_size=CFG.batch_size,
    per_device_eval_batch_size=CFG.batch_size,
    num_train_epochs=CFG.epochs,
    weight_decay=CFG.weight_decay,  
    load_best_model_at_end=True,
    metric_for_best_model=metric_name,
    save_total_limit=1
)
    
for fold in range(CFG.num_fold):
    
    tr_data = train_df[train_df['fold']!=fold].reset_index(drop=True)
    va_data = train_df[train_df['fold']==fold].reset_index(drop=True)
    tr_dataset = TrainDataset(tr_data)
    va_dataset = TrainDataset(va_data)
    
    trainer = Trainer(
        model,
        args,
        train_dataset=tr_dataset,
        eval_dataset=va_dataset,
        tokenizer=tokenizer,
        compute_metrics=compute_metrics
    )
    
    trainer.train()
    shutil.rmtree(f"./drive/MyDrive/Kaggle/USPatent/{model_name}-finetuned")
    trainer.save_model(f"./drive/MyDrive/Kaggle/USPatent/{model_name}-finetuned-{fold}")
    
    outputs = trainer.predict(va_dataset)
    predictions = outputs.predictions.reshape(-1)
    va_data['preds'] = predictions
    oof_df = pd.concat([oof_df, va_data])

Downloading:   0%|          | 0.00/671 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/418M [00:00<?, ?B/s]

Some weights of the model checkpoint at AI-Growth-Lab/PatentSBERTa were not used when initializing MPNetForSequenceClassification: ['pooler.dense.weight', 'pooler.dense.bias']
- This IS expected if you are initializing MPNetForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing MPNetForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of MPNetForSequenceClassification were not initialized from the model checkpoint at AI-Growth-Lab/PatentSBERTa and are newly initialized: ['classifier.dense.weight', 'classifier.dense.bias', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be ab

Epoch,Training Loss,Validation Loss,Pearson,Spearmanr
1,No log,0.027366,0.790318,0.764885
2,0.036700,0.023285,0.814669,0.791882
3,0.023300,0.022615,0.821388,0.799390
4,0.018700,0.022795,0.825783,0.804533
5,0.016200,0.022994,0.827244,0.807018


***** Running Evaluation *****
  Num examples = 7295
  Batch size = 64
Saving model checkpoint to ./drive/MyDrive/Kaggle/USPatent/PatentSBERTa-finetuned/checkpoint-456
Configuration saved in ./drive/MyDrive/Kaggle/USPatent/PatentSBERTa-finetuned/checkpoint-456/config.json
Model weights saved in ./drive/MyDrive/Kaggle/USPatent/PatentSBERTa-finetuned/checkpoint-456/pytorch_model.bin
tokenizer config file saved in ./drive/MyDrive/Kaggle/USPatent/PatentSBERTa-finetuned/checkpoint-456/tokenizer_config.json
Special tokens file saved in ./drive/MyDrive/Kaggle/USPatent/PatentSBERTa-finetuned/checkpoint-456/special_tokens_map.json
***** Running Evaluation *****
  Num examples = 7295
  Batch size = 64
Saving model checkpoint to ./drive/MyDrive/Kaggle/USPatent/PatentSBERTa-finetuned/checkpoint-912
Configuration saved in ./drive/MyDrive/Kaggle/USPatent/PatentSBERTa-finetuned/checkpoint-912/config.json
Model weights saved in ./drive/MyDrive/Kaggle/USPatent/PatentSBERTa-finetuned/checkpoint-912/pyto

...

Epoch,Training Loss,Validation Loss,Pearson,Spearmanr
1,No log,0.008927,0.938321,0.926313
2,0.012100,0.009680,0.940381,0.927404
3,0.009700,0.009817,0.939229,0.926223
4,0.008100,0.008160,0.940138,0.925932
5,0.007000,0.008771,0.940147,0.926184


***** Running Evaluation *****
  Num examples = 7294
  Batch size = 64
Saving model checkpoint to ./drive/MyDrive/Kaggle/USPatent/PatentSBERTa-finetuned/checkpoint-456
Configuration saved in ./drive/MyDrive/Kaggle/USPatent/PatentSBERTa-finetuned/checkpoint-456/config.json
Model weights saved in ./drive/MyDrive/Kaggle/USPatent/PatentSBERTa-finetuned/checkpoint-456/pytorch_model.bin
tokenizer config file saved in ./drive/MyDrive/Kaggle/USPatent/PatentSBERTa-finetuned/checkpoint-456/tokenizer_config.json
Special tokens file saved in ./drive/MyDrive/Kaggle/USPatent/PatentSBERTa-finetuned/checkpoint-456/special_tokens_map.json
***** Running Evaluation *****
  Num examples = 7294
  Batch size = 64
Saving model checkpoint to ./drive/MyDrive/Kaggle/USPatent/PatentSBERTa-finetuned/checkpoint-912
Configuration saved in ./drive/MyDrive/Kaggle/USPatent/PatentSBERTa-finetuned/checkpoint-912/config.json
Model weights saved in ./drive/MyDrive/Kaggle/USPatent/PatentSBERTa-finetuned/checkpoint-912/pyto

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 29179
  Num Epochs = 5
  Instantaneous batch size per device = 64
  Total train batch size (w. parallel, distributed & accumulation) = 64
  Gradient Accumulation steps = 1
  Total optimization steps = 2280


Epoch,Training Loss,Validation Loss,Pearson,Spearmanr
1,No log,0.007681,0.949054,0.936213
2,0.009900,0.007797,0.948977,0.935770
3,0.008000,0.007750,0.948585,0.933662
4,0.006700,0.007631,0.948573,0.933699
5,0.005600,0.007238,0.949948,0.933890


***** Running Evaluation *****
  Num examples = 7294
  Batch size = 64
Saving model checkpoint to ./drive/MyDrive/Kaggle/USPatent/PatentSBERTa-finetuned/checkpoint-456
Configuration saved in ./drive/MyDrive/Kaggle/USPatent/PatentSBERTa-finetuned/checkpoint-456/config.json
Model weights saved in ./drive/MyDrive/Kaggle/USPatent/PatentSBERTa-finetuned/checkpoint-456/pytorch_model.bin
tokenizer config file saved in ./drive/MyDrive/Kaggle/USPatent/PatentSBERTa-finetuned/checkpoint-456/tokenizer_config.json
Special tokens file saved in ./drive/MyDrive/Kaggle/USPatent/PatentSBERTa-finetuned/checkpoint-456/special_tokens_map.json
***** Running Evaluation *****
  Num examples = 7294
  Batch size = 64
Saving model checkpoint to ./drive/MyDrive/Kaggle/USPatent/PatentSBERTa-finetuned/checkpoint-912
Configuration saved in ./drive/MyDrive/Kaggle/USPatent/PatentSBERTa-finetuned/checkpoint-912/config.json
Model weights saved in ./drive/MyDrive/Kaggle/USPatent/PatentSBERTa-finetuned/checkpoint-912/pyto

In [13]:
predictions = oof_df['preds'].values
label = oof_df['score'].values
eval_pred = predictions, label
compute_metrics(eval_pred)

{'pearson': 0.9076119788629591, 'spearmanr': 0.8927606983401641}

In [14]:
oof_df.to_csv('oof_df.csv')

In [18]:
oof_df

,id,anchor,target,context,score,code,title,section,class,subclass,group,main_group,context_text,text,fold,preds
0,54c1e3b9184cb5b6,abatement,forest region,A47,0.00,A47,FURNITURE; DOMESTIC ARTICLES OR APPLIANCES; CO...,A,47.0,NaN,NaN,NaN,HUMAN NECESSITIES. FURNITURE; DOMESTIC ARTICLE...,abatement[sep]forest region[sep]human necessit...,0,0.001302
1,ef2d4c2e6bbb208d,abatement,mixing core materials,A47,0.25,A47,FURNITURE; DOMESTIC ARTICLES OR APPLIANCES; CO...,A,47.0,NaN,NaN,NaN,HUMAN NECESSITIES. FURNITURE; DOMESTIC ARTICLE...,abatement[sep]mixing core materials[sep]human ...,0,0.209500
2,cc96541d4987b399,abatement,rent abatement,A47,0.00,A47,FURNITURE; DOMESTIC ARTICLES OR APPLIANCES; CO...,A,47.0,NaN,NaN,NaN,HUMAN NECESSITIES. FURNITURE; DOMESTIC ARTICLE...,abatement[sep]rent abatement[sep]human necessi...,0,0.048777
3,a8c9e9f37d4d836a,abatement,tax abatement,A47,0.00,A47,FURNITURE; DOMESTIC ARTICLES OR APPLIANCES; CO...,A,47.0,NaN,NaN,NaN,HUMAN NECESSITIES. FURNITURE; DOMESTIC ARTICLE...,abatement[sep]tax abatement[sep]human necessit...,0,-0.005800
4,44cbf2111c2c64d2,adhesive mounting,adhesive attachment,A47,0.50,A47,FURNITURE; DOMESTIC ARTICLES OR APPLIANCES; CO...,A,47.0,NaN,NaN,NaN,HUMAN NECESSITIES. FURNITURE; DOMESTIC ARTICLE...,adhesive mounting[sep]adhesive attachment[sep]...,0,0.606328
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7289,9e9fd37c14af9ddb,opposing walls,opposite length,F26,0.25,F26,DRYING,F,26.0,NaN,NaN,NaN,MECHANICAL ENGINEERING; LIGHTING; HEATING; WEA...,opposing walls[sep]opposite length[sep]mechani...,4,0.377219
7290,b0de03de82f17135,undulation,lateral undulation,B31,0.00,B31,"MAKING ARTICLES OF PAPER, CARDBOARD OR MATERIA...",B,31.0,NaN,NaN,NaN,PERFORMING OPERATIONS; TRANSPORTING. MAKING AR...,undulation[sep]lateral undulation[sep]performi...,4,-0.007944
7291,cd209e0b7f9754ef,undulation,stationary,B31,0.25,B31,"MAKING ARTICLES OF PAPER, CARDBOARD OR MATERIA...",B,31.0,NaN,NaN,NaN,PERFORMING OPERATIONS; TRANSPORTING. MAKING AR...,undulation[sep]stationary[sep]performing opera...,4,0.255608
7292,9ceaa1f06ffcdc14,undulation,undulating,B31,1.00,B31,"MAKING ARTICLES OF PAPER, CARDBOARD OR MATERIA...",B,31.0,NaN,NaN,NaN,PERFORMING OPERATIONS; TRANSPORTING. MAKING AR...,undulation[sep]undulating[sep]performing opera...,4,1.029155


# Hyperparameter search

In [15]:
! pip install optuna
! pip install ray[tune]

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 308 kB 648 kB/s 
     |████████████████████████████████| 81 kB 5.2 MB/s 
     |████████████████████████████████| 210 kB 9.7 MB/s 
     |████████████████████████████████| 78 kB 4.9 MB/s 
     |████████████████████████████████| 146 kB 11.5 MB/s 
     |████████████████████████████████| 49 kB 4.8 MB/s 
     |████████████████████████████████| 112 kB 19.7 MB/s 
  Created wheel for pyperclip: filename=pyperclip-1.8.2-py3-none-any.whl size=11137 sha256=d3678983b6166406ba5572d418dce3e0b5b5b6710b29ec61e79e68f10c8649c4
  Stored in directory: /root/.cache/pip/wheels/9f/18/84/8f69f8b08169c7bae2dde6bd7daf0c19fca8c8e500ee620a28
Successfully built pyperclip
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 53.2 MB 287 kB/s 
     |████████████████████████████████| 8.8 MB 63.2 MB

In [ ]:
def model_init():
    return AutoModelForSequenceClassification.from_pretrained(model_checkpoint, num_labels=num_labels)

In [ ]:
trainer = Trainer(
    model_init=model_init,
    args=args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [ ]:
best_run = trainer.hyperparameter_search(n_trials=10, direction="maximize")

In [ ]:
best_run

In [ ]:
for n, v in best_run.hyperparameters.items():
    setattr(trainer.args, n, v)

trainer.train()
trainer.save_model('./drive/MyDrive/Kaggle/USPatent/Models')